In [ ]:
import csv
import logging
import requests
from datetime import datetime, timedelta
from time import sleep
import os
import shutil
import pandas as pd
logging.basicConfig(level=logging.DEBUG)

# Base URL
base_url = "https://admin.ezsend.io/admin/api/v1"

# Authentication endpoint
auth_endpoint = f"https://admin.ezsend.io/admin/api/session"

In [ ]:
def authenticate(username, password):
    auth_data = {"username": username, "password": password}
    response = requests.post(auth_endpoint, json=auth_data)
    if response.status_code == 200:
        auth_token = response.json().get("authtoken")
        print("Authentication successful. Auth token:", auth_token)
        return auth_token
    else:
        print("Authentication failed:", response.status_code)
        print("Response content:", response.content)
        return None

In [ ]:
def fetch_data(endpoint, auth_token):
    headers = {"X-Authtoken": auth_token, "Accept": "application/json"}  # Use Accept header instead of Content-Type
    max_retries = 5
    retries = 0
    
    while retries < max_retries:
        response = requests.get(endpoint, headers=headers)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 502:
            retries += 1
            print(f"502 error encountered. Retrying {retries}/{max_retries} in 5 seconds...")
            sleep(30)
        else:
            print("Failed to fetch data from endpoint:", endpoint)
            print("Response status code:", response.status_code)
            print("Response content:", response.content)
            return None
    
    print("Maximum retries reached. Unable to fetch data.")
    return None

In [ ]:
# Function to generate endpoints with start and end dates
def generate_endpoint(endpoint_name, start_date=None, end_date=None, scheduled_start=None, scheduled_end=None):
    # Dictionary to map endpoint names to their display names
    endpoint_display_names = {
        "sms": "SMS_SC_SS_Report",
        "segment": "SMS_SC_SS_Segments",
        "offer": "SMS_SC_SS_Offers",
        "smsjob": "jobs",
        "smsreport": "creativestats", # SMS Offer Report
        "apikey": "apikeyreport",
        "smsflowreport": "SMS_SC_SS_Flows"
    }
    
    # Base endpoint URL
    endpoint = f"{base_url}/{endpoint_name}?offset=0&limit=-1"
    
    # Sorting configuration for each endpoint
    sort_config = {
        "sms": "&sort_column=shortcode&sort_order=descending",
        "segment": "&sort_column=id&sort_order=descending",
        "offer": "&sort_column=id&sort_order=ascending",
        "smsjob": "&sort_column=status&sort_order=ascending",
        "smsreport": "&sort_column=job_tstamp&sort_order=ascending",
        "apikey": "&sort_column=name&sort_order=ascending",
        "smsflowreport": "&sort_column=count_deliver&sort_order=descending"
    }
    
    # Add sorting configuration
    endpoint += sort_config.get(endpoint_name, "")
    
    # Filter configuration for Jobs endpoint
    if endpoint_name == "smsjob":
        scheduled_start = start_date if scheduled_start else None
        scheduled_end = end_date if scheduled_end else None
        if scheduled_start:
            endpoint += f"&filter=scheduled_tstamp_date_start%7Cgreater_or_equal%7C{scheduled_start}"
        if scheduled_end:
            endpoint += f"&filter=scheduled_tstamp_date_end%7Cless_than%7C{scheduled_end}"
    
    # Start and end date configuration
    if start_date and end_date:
        endpoint += f"&start={start_date}&end={end_date}"
    
    return endpoint

# Function to save data to CSV file
def save_to_csv(data, filename):
    if data:
        # Get all unique keys across the data dictionaries
        fieldnames = set().union(*(d.keys() for d in data))
        
        with open(filename, mode='w', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(data)
        
        print(f"Data saved to {filename}")
    else:
        print("No data received. Unable to save to CSV.")

In [ ]:
if datetime.today().isoweekday() == 1: # Check if it is Monday
    start_date = (datetime.today() - timedelta(days=3)).strftime('%Y-%m-%d') # If Monday: start_date = Friday
else:
    start_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')

end_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')
report_date = (datetime.today() - timedelta(days=1)).strftime('%Y%m%d')
today_date = datetime.today().strftime('%Y-%m-%d')

# Convert start_date and end_date back to datetime.date objects
start_date_dt = datetime.strptime(start_date, '%Y-%m-%d').date()
end_date_dt = datetime.strptime(end_date, '%Y-%m-%d').date()

# Generate the list of dates
if datetime.today().isoweekday() == 1:
    date_list = [(start_date_dt + timedelta(days=x)).strftime("%Y%m%d") for x in range((end_date_dt - start_date_dt).days + 1)]
    # Print the list of dates
    print(date_list)

print("Start date:", start_date)
print("End date:", end_date)
print("Report date:", report_date)
username = "rxmarketing_andrew"
password = "t37yA^27to@U666Dcodp"

In [ ]:
# Authenticate
auth_token = authenticate(username, password)

if auth_token:
    # Generate endpoints with appropriate parameters
    endpoints = [
        ("SMS_SC_SS_Report", generate_endpoint("sms", start_date, end_date)),  # SMS Report
        ("SMS_SC_SS_Segments", generate_endpoint("segment")),  # Segments
        ("SMS_SC_SS_Offers", generate_endpoint("offer")),  # Offer List
        ("jobs", generate_endpoint("smsjob", start_date, end_date)),  # Jobs
        ("creativestats", generate_endpoint("smsreport", start_date, end_date)),  # SMS CreativeStats / SMS Offer Report 
        ("apikeyreport", generate_endpoint("apikey")),  # Apikey Report
        ("SMS_SC_SS_Flows", generate_endpoint("smsflowreport", start_date, end_date))  # SMS Flow Report
    ]

# Fetch data from endpoints and save to CSV files
for endpoint_name, endpoint in endpoints:
    if (endpoint_name == 'SMS_SC_SS_Report') & (start_date != end_date):
        daylist = pd.date_range(start_date, end_date, freq='D').strftime('%Y-%m-%d')
        for i in daylist:
            start_date_sms_report = i
            end_date_sms_report = i
            endpoints = [ ("SMS_SC_SS_Report", generate_endpoint("sms", start_date_sms_report, end_date_sms_report))]  # SMS Flow Report
            print(endpoints)
            data_dict = fetch_data(endpoints[0][1], auth_token)
            if data_dict and 'data' in data_dict:
                data = data_dict['data']
                end_date_format = datetime.strptime(end_date_sms_report, "%Y-%m-%d").strftime('%Y%m%d')
                filename = f"{endpoint_name}{end_date_format}.csv"
                save_to_csv(data, filename)
                print(f"Data saved to {filename}")
                sleep(30)
    
    else: 
        data_dict = fetch_data(endpoint, auth_token)
        if data_dict and 'data' in data_dict:
            data = data_dict['data']
                
            #filename = f"{endpoint_name}_{start_date}_{end_date}.csv"
            
            if endpoint_name == 'SMS_SC_SS_Report':
                end_date_format = datetime.strptime(end_date, "%Y-%m-%d").strftime('%Y%m%d')
                filename = f"{endpoint_name}{end_date_format}.csv"
            elif endpoint_name == 'SMS_SC_SS_Segments':
                end_date_format = datetime.strptime(today_date, "%Y-%m-%d").strftime('%Y%m%d')
                filename = f"{endpoint_name}{end_date_format}.csv"
            elif endpoint_name == 'apikeyreport':
                #end_date_format = datetime.strptime(end_date, '%Y-%m-%d')
                filename = f"{endpoint_name}{end_date}.csv"
            else:
                filename = f"{endpoint_name}.csv"
            
            save_to_csv(data, filename)
            print(f"Data saved to {filename}")
            sleep(30)
        else:
            print(f"Failed to fetch data for {endpoint_name}") 

In [ ]:
# Save files to a folder

new_dir = f'SMS_Data_Files_{end_date[5:7]}.{end_date[8:10]}' # Specify the path for the new directory
os.mkdir(new_dir)
print(f"Directory {new_dir} created")

# Enter current directory path
source_dir = '/Users/nathanohashi/Documents/VSCode/automated_daily_download_process' # Working Directory

all_csv = [file for file in os.listdir(source_dir) if file.endswith('.csv')]

for file in all_csv:
    src_path = os.path.join(source_dir, file)
    dst_path = os.path.join(new_dir, file)
    shutil.move(src_path, dst_path)